In [1]:
import pandas as pd
from datasets import load_abundance_data, load_markers_data
from Parameter_Sharing.train_param_sharing import tune_shared_encoder
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
dfs=load_abundance_data()
#markers=load_markers_data()

In [3]:
def tune_across_shared_datasets(dataset_list,
                                model_name,
                                tuning_func,
                                path, 
                                is_marker):
    if os.path.isfile(path):
        raise(ValueError('Will not run function and potentially overwrite important results! Either delete or move the file first'))
    
    # initialize csv to store results in
    results=pd.DataFrame(columns =  ['Model Type', 'Data Type', 'Dataset', 'hyperparams', 'AUC'])
    results.to_csv(path, index=False)
    # function to loop the tuning functions across each dataset
    for i in range(len(dataset_list)):
        datas = dataset_list[i:] + dataset_list[:i]
        print(datas[0].index[0])
        results=tuning_func(datas, 
                            model_name=model_name,
                            is_marker=is_marker)
        results = pd.DataFrame([a if type(a)==str 
                                 else [b for c,b in a.items()] if type(a) == dict 
                                 else a[0]['objective'] if type(a)==tuple
                                 else a
                                 for a in results]).T
        results.columns=['Model Type', 'Data Type', 'Dataset', 'hyperparams', 'AUC']
        pd.concat( [pd.read_csv(path),results], axis=0 ).to_csv(path, index=False)
        print('Dataset #'+str(i)+' complete!')

In [4]:
def get_shared_taxa_dfs(dfs):
    disease_map=pd.DataFrame( ['Zeller_fecal_colorectal_cancer',
                             'Quin_gut_liver_cirrhosis',
                             'metahit',
                             't2dmeta_long',
                             'WT2D',
                             'Chatelier_gut_obesity', 
                              't2dmeta_short'], 
                 ['Colorectal', 'Cirrhosis', 'IBD', 'C-T2D', 'EW-T2D', 'Obesity', 'C-T2D'] 
                ).reset_index()

    disease_map.columns = ['clean', 'raw']
    cols = []#['disease']
    [cols.append( df.columns[df.columns.str.contains('k__')] ) for df in dfs]
    all_dfs = pd.concat( [dfs[i][cols[i]].astype(float)# if i > 0
                          #else dfs[i][cols[i]] 
                          for i in range(len(dfs))], axis = 1).fillna(0)
    all_dfs = all_dfs.T.groupby(['dataset_name']).sum().T
    all_dfs['disease'] = pd.concat( [df['disease'] for df in dfs] )

    all_dfs['dataset'] = all_dfs.index.str.split('.').str[0]
    all_dfs = all_dfs[['dataset', 'disease']+list(all_dfs.columns[:-2])]
    all_dfs = all_dfs.reset_index(drop=True)
    all_dfs = all_dfs.fillna(0)
    all_dfs=disease_map.merge(all_dfs, left_on='raw', right_on='dataset').drop(['raw', 'dataset'], axis=1)
    all_dfs.columns=['dataset'] + list(all_dfs.columns[1:])
    
    names = ['Obesity', 'Colorectal', 'IBD', 'EW-T2D','C-T2D', 'Cirrhosis']
    
    datasets = [all_dfs.loc[all_dfs.dataset==a]\
            for a in names]
    
    return({names[i]:datasets[i] for i in range(len(names))} )

In [5]:
all_datasets=get_shared_taxa_dfs(dfs)

In [6]:
names = ['Obesity', 'Colorectal', 'IBD', 'EW-T2D','C-T2D', 'Cirrhosis']

In [7]:
#creating different groups for param sharing
groups = {'EU': ['Obesity', 'Colorectal', 'IBD', 'EW-T2D'],
          'CH':['C-T2D', 'Cirrhosis'], 
          'T2D':['EW-T2D','C-T2D'],
          'Colon':['Colorectal','IBD']
         }


models = ['VAE', 'DAE', 'SAE'] # different models we're considering in this section

In [8]:
i = 0
for g in groups:
    for mod in models:
        if i > 1:
            subset_names = groups[g]
            ds = [all_datasets[s] for s in subset_names]
            path = 'sharing_results/' + g + '_' + mod + '.csv'
            tune_across_shared_datasets(ds,
                                    mod,
                                    tune_shared_encoder,
                                    path, 
                                    is_marker=False)
        
        i += 1

[INFO 04-12 23:06:10] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-12 23:06:10] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-12 23:06:10] ax.service.managed_loop: Running optimization trial 1...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores


830


[INFO 04-12 23:06:19] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-12 23:06:19] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-12 23:06:19] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-12 23:06:19] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-12 23:06:19] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-12 23:06:19] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-12 23:06:20] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-12 23:06:20] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-12 23:06:20] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-12 23:06:20] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-12 23:06:20] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-12 23:06:20] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-12 23:06:21] ax.service.ma

[INFO 04-12 23:08:32] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-12 23:08:35] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-12 23:08:37] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-12 23:08:39] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-12 23:08:47] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-12 23:08:50] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-12 23:08:54] ax.service.managed_loop: Running optimization trial 8...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
[INFO 04-12 23:09:10] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-12 23:09:10] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-12 23:09:10] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-12 23:09:14] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-12 23:09:22] ax.service.managed

Dataset #0 complete!
0


...

[INFO 04-13 09:57:30] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 09:57:30] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 09:57:30] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-13 09:57:30] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-13 09:57:30] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-13 09:57:30] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-13 09:57:30] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-13 09:57:31] ax.service.managed_loop: Running optimization trial 9...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
[INFO 04-13 09:57:40] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 09:57:40] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-13 09:57:40] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-13 09:57:40] ax.service.managed

Dataset #0 complete!
353


[INFO 04-13 09:58:00] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 09:58:00] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-13 09:58:00] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-13 09:58:03] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-13 09:58:08] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-13 09:58:14] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 09:58:17] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 09:58:18] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-13 09:58:24] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-13 09:58:28] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-13 09:58:32] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-13 09:58:36] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-13 09:58:41] ax.service.ma

[INFO 04-13 10:01:05] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 10:01:05] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 10:01:05] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-13 10:01:05] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-13 10:01:05] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-13 10:01:05] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-13 10:01:05] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-13 10:01:05] ax.service.managed_loop: Running optimization trial 9...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
[INFO 04-13 10:01:15] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 10:01:15] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-13 10:01:15] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-13 10:01:20] ax.service.managed

Dataset #1 complete!


In [9]:
# running combining everything
for mod in models:
    subset_names = names
    ds = [all_datasets[s] for s in subset_names]
    path = 'sharing_results/' + 'ALL' + '_' + mod + '.csv'
    tune_across_shared_datasets(ds,
                                mod,
                                tune_shared_encoder,
                                path, 
                                is_marker=False)


[INFO 04-13 10:02:36] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 10:02:36] ax.service.managed_loop: Started full optimization with 15 steps.
[INFO 04-13 10:02:36] ax.service.managed_loop: Running optimization trial 1...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores


830


[INFO 04-13 10:02:47] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 10:02:47] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-13 10:02:47] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-13 10:02:47] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-13 10:02:47] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-13 10:02:47] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 10:02:47] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 10:02:47] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-13 10:02:47] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-13 10:02:48] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-13 10:02:48] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-13 10:02:48] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-13 10:02:48] ax.service.ma

GPU available: False, used: False
TPU available: None, using: 0 TPU cores
[INFO 04-13 10:05:53] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 10:05:53] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-13 10:05:53] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-13 10:05:53] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-13 10:05:53] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-13 10:05:53] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 10:05:53] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 10:05:53] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-13 10:05:53] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-13 10:05:53] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-13 10:05:53] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-13 10:05:53] ax.service.managed_

[INFO 04-13 10:09:10] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-13 10:09:10] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 10:09:10] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 10:09:10] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-13 10:09:10] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-13 10:09:10] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-13 10:09:10] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-13 10:09:10] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-13 10:09:10] ax.service.managed_loop: Running optimization trial 15...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
[INFO 04-13 10:09:19] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 10:09:19] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-13 10:09:19] ax.service.manage

...

[INFO 04-13 12:34:49] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-13 12:34:50] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-13 12:34:50] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 12:34:50] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 12:34:50] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-13 12:34:51] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-13 12:34:51] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-13 12:34:51] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-13 12:34:51] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-13 12:34:52] ax.service.managed_loop: Running optimization trial 9...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
[INFO 04-13 12:35:03] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 12:35:03] ax.service.managed_loop: St

Dataset #4 complete!
121


[INFO 04-13 12:36:29] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 12:36:29] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-13 12:36:29] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-13 12:36:36] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-13 12:36:38] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-13 12:36:41] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 12:36:46] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 12:36:47] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-13 12:36:50] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-13 12:36:59] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-13 12:37:06] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-13 12:37:07] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-13 12:37:08] ax.service.ma

[INFO 04-13 12:41:31] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 12:41:38] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 12:41:42] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-13 12:41:43] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-13 12:41:47] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-13 12:41:49] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-13 12:41:50] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-13 12:41:52] ax.service.managed_loop: Running optimization trial 9...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
[INFO 04-13 12:42:04] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 12:42:04] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-13 12:42:04] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-13 12:42:04] ax.service.managed

Dataset #5 complete!


In [10]:
markers=load_markers_data()

In [11]:
def get_shared_taxa_markers(dfs):
    disease_map=pd.DataFrame( ['Zeller_fecal_colorectal_cancer',
                             'Quin_gut_liver_cirrhosis',
                             'metahit',
                             't2dmeta_long',
                             'WT2D',
                             'Chatelier_gut_obesity', 
                              't2dmeta_short'], 
                 ['Colorectal', 'Cirrhosis', 'IBD', 'C-T2D', 'EW-T2D', 'Obesity', 'C-T2D'] 
                ).reset_index()

    disease_map.columns = ['clean', 'raw']
    marker_cols = []
    [marker_cols.append( df.columns[df.columns.str.contains('gi[|]')] ) for df in markers]

    all_markers = pd.concat( [markers[i][marker_cols[i]].astype(float) for i in range(len(markers))], axis = 1).fillna(0)
    all_markers = all_markers.T.groupby('dataset_name').sum().T
    all_markers['disease'] = pd.concat( [df['disease'] for df in markers] )
    all_markers['dataset'] = all_markers.index.str.split('.').str[0]
    all_markers = all_markers[['dataset', 'disease']+list(all_markers.columns[:-2])]
    all_markers = all_markers.reset_index(drop=True)
    all_markers = all_markers.fillna(0)
    all_markers=disease_map.merge(all_markers, left_on='raw', right_on='dataset').drop(['raw', 'dataset'], axis=1)
    all_markers.columns=['dataset'] + list(all_markers.columns[1:])
    names = ['Obesity', 'Colorectal', 'IBD', 'EW-T2D','C-T2D', 'Cirrhosis']
    
    datasets = [all_markers.loc[all_markers.dataset==a]\
            for a in names]
    
    return({names[i]:datasets[i] for i in range(len(names))} )



In [12]:
all_marker_datasets=get_shared_taxa_markers(markers)

In [14]:
i = 0
for g in groups:
    for mod in models:
        if i > 3:
            subset_names = groups[g]
            ds = [all_marker_datasets[s] for s in subset_names]
            path = 'sharing_results/' + g + '_' + mod + '_markers.csv'
            tune_across_shared_datasets(ds,
                                    mod,
                                    tune_shared_encoder,
                                    path, 
                                    is_marker=True)
        
        i += 1

463


[INFO 04-13 21:37:17] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 21:37:17] ax.service.managed_loop: Started full optimization with 15 steps.
[INFO 04-13 21:37:17] ax.service.managed_loop: Running optimization trial 1...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to

Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
[INFO 04-13 21:45:03] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-13 21:45:03] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-13 21:45:03] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-13 21:45:08] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-13 21:45:16] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-13 21:45:20] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-13 21:45:26] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-13 21:45:30] ax.service.managed_

Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has 

Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has 

...

Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has 

[INFO 04-14 11:38:46] ax.service.managed_loop: Running optimization trial 6...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or mi

Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
[INFO 04-14 11:44:00] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-14 11:44:00] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-14 11:44:00] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-14 11:44:00] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-14 11:44:06] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-14 11:44:06] ax.service.managed_loop: Running optimization t

Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has 

Dataset #1 complete!


In [ ]:
i = 0
for g in groups:
    for mod in models:
        if i <= 1:
            subset_names = groups[g]
            ds = [all_marker_datasets[s] for s in subset_names]
            path = 'sharing_results/' + g + '_' + mod + '_markers.csv'
            tune_across_shared_datasets(ds,
                                    mod,
                                    tune_shared_encoder,
                                    path, 
                                    is_marker=True)
        
        i += 1

830


[INFO 04-14 19:13:15] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-14 19:13:15] ax.service.managed_loop: Started full optimization with 15 steps.
[INFO 04-14 19:13:15] ax.service.managed_loop: Running optimization trial 1...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to

Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has 

[INFO 04-14 19:37:32] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-14 19:37:36] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-14 19:37:38] ax.service.managed_loop: Running optimization trial 6...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
[INFO 04-14 19:40:00] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-14 19:40:00] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-14 19:40:00] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-14 19:40:00] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-14 19:40:00] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-14 19:40

[INFO 04-14 19:54:16] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-14 19:54:16] ax.service.managed_loop: Running optimization trial 9...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will continue...
Trainer was signaled to stop but required minimum epochs (50) or minimum steps (None) has not been met. Training will

In [ ]:
# running combining everything
for mod in models:
    subset_names = names
    ds = [all_marker_datasets[s] for s in subset_names]
    path = 'sharing_results/' + 'ALL' + '_' + mod + '_markers.csv'
    tune_across_shared_datasets(ds,
                                mod,
                                tune_shared_encoder,
                                path, 
                                is_marker=False)